In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import display
import numpy as np

# %matplotlib inline

from pathlib import Path


In [2]:
def collect_one(run, csv_folder, test_config='general'):
    #if run.name == 'baseline':
    parameters = {p.split('=')[0]: p.split('=')[1] for p in run.as_posix().split('/') if "=" in p}
    # else:
    #     with open(run / 'config.json', 'r') as f:
    #         cfg = json.load(f)
            
    #     run_dir = cfg["run_dir"]
    #     parameters = {p.split('=')[0]: p.split('=')[1] for p in run_dir.split('/') if "=" in p}

    data = []
    for yamlf in (run / 'inference' / csv_folder).rglob('*.csv'):
        test_config_name = yamlf.parent.stem
        if test_config not in yamlf.parent.stem:
            continue
        df = pd.read_csv(yamlf)
        # keep only the last line. This is because until this commit, we were appending the logs and not overriding them, so the last line is the one we want
        df = df.tail(1)
        df['tok_position_inference'] = 'beginning' if 'tok_beginning' in test_config_name else 'in_place' if 'tok_in_place' in test_config_name else None
        df['inference_config'] = test_config_name.split('_')[-1] if 'conf' in test_config_name else None
        if run.name == 'baseline':
            df['model'] = 'clip_original'
        data.append(df)
    
    data = pd.concat(data)
    # data.columns.names = ['type', 'metric']
    # data.sort_values(by=['type', 'metric'], axis=1, inplace=True)
    data.drop(columns=['epoch', 'step'], inplace=True)
    
    if data.empty:
        print(f'Pred folder is empty: {csv_folder}')
    
    for k, v in parameters.items():
        data[k] = v
    
    return data

def collect_all(root, csv_folder, test_config='general'):
    root = Path(root)
    metrics = [collect_one(csvf.parents[1], csvf.name, test_config=test_config) for csvf in list(root.rglob(csv_folder))]
    metrics = pd.concat(metrics, ignore_index=True)
    return metrics

default_fields_dict = {
    'r1': lambda x: u"{:.1f}".format(x),
    'r5': lambda x: u"{:.1f}".format(x),
    'r10': lambda x: u"{:.1f}".format(x),
    'meanr': lambda x: u"{:.1f}".format(x),
    'medr': lambda x: int(x),
    'spice': lambda x: u"{:.3f}".format(x),
    'spacy': lambda x: u"{:.3f}".format(x),
}
def render_to_latex(metrics, rename_func=default_fields_dict, **latex_kwargs):
    m = metrics.copy()
    # renaming
    for col, lambda_fn in rename_func.items():
        m[col] = m[col].apply(lambda_fn)
    # m = m.applymap(lambda x: u"{:.2f}".format(x))
    ltex = m.style.to_latex(
        **latex_kwargs
    )
    return ltex

In [3]:
# Compute metrics for each detected run

def summarize_metrics(
        metrics, 
        dataset=None, 
        model=None,
        translator=None,
        tok_position=None, 
        training_setup=None, 
        inference_config=None,
        loss=None, 
        learning_rate=None, 
        finetuning=None, 
        drop_i2t=True,
        decimal_places=3):
    
    if dataset is not None:
        metrics = metrics[(metrics['data'] == dataset)]
        metrics.drop(columns="data", inplace=True)

    # TODO: as of now, there is only one split seed.
    # In the future, we would have to average among different splits
    # metrics.drop(columns="split_seed", inplace=True)

    id_vars = ['data', 'model', 'translator', 'tok_position', 'training-setup', 'loss', 'lr', 'finetuning', 'inference_config']

    if translator is not None:
        metrics = metrics[metrics['translator'].isin(learning_rate)]
        if len(translator) == 1:
            metrics.drop(columns="translator", inplace=True)
            id_vars.remove('translator')
    if model is not None:
        metrics = metrics[metrics['model'].isin(learning_rate)]
        if len(model) == 1:
            metrics.drop(columns="model", inplace=True)
            id_vars.remove('model')
    if learning_rate is not None:
        metrics = metrics[metrics['lr'].isin(learning_rate)]
        if len(learning_rate) == 1:
            metrics.drop(columns="lr", inplace=True)
            id_vars.remove('lr')
    if finetuning is not None:
        metrics = metrics[metrics['finetuning'].isin(finetuning)]
        if len(finetuning) == 1:
            metrics.drop(columns="finetuning", inplace=True)
            id_vars.remove('finetuning')
    if tok_position is not None:
        metrics = metrics[metrics['tok_position'].isin(tok_position)]
        if len(tok_position) == 1:
            metrics.drop(columns="tok_position", inplace=True)
            id_vars.remove('tok_position')
    if training_setup is not None:
        metrics = metrics[metrics['training-setup'].isin(training_setup)]
        if len(training_setup) == 1:
            metrics.drop(columns="training-setup", inplace=True)
            id_vars.remove('training-setup')
    if loss is not None:
        metrics = metrics[metrics['loss'].isin(loss)]
        if len(loss) == 1:
            metrics.drop(columns="loss", inplace=True)
            id_vars.remove('loss')
    if inference_config is not None:
        metrics = metrics[metrics['inference_config'].isin(inference_config)]
        if len(inference_config) == 1:
            metrics.drop(columns="inference_config", inplace=True)
            id_vars.remove('inference_config')

    if drop_i2t:
        # remove columns containing i2t in the name of the second level of the multiindex
        metrics = metrics.loc[:, ~metrics.columns.str.contains('i2t')]

    # round to given decimal places
    metrics = metrics.round(decimal_places)

    metrics.set_index(id_vars, inplace=True)
    # split into different dataframes, one for each column (first level of the multiindex)
    # column_types = list(metrics.columns.get_level_values(0).unique())
    # metrics = {c: metrics.loc[:, c].copy() for c in column_types}
    # {k: v.columns.set_names(k, inplace=True) for k, v in metrics.items()}
    return metrics

In [4]:
# rename content of the table
def rename_fn(v):
    mapping = {'ContrastiveFixed': 'Triplet',
               'InfoNCELoss': 'InfoNCE'}
    if v in mapping:
        return mapping[v]
    return v

def render_to_latex(metrics, rename_func=default_fields_dict, **latex_kwargs):
    m = metrics.copy()
     # make bold the best values

    # Custom function to highlight the maximum value in each group
    def highlight_best(data):
        attr = 'font-weight: bold'
        result = pd.DataFrame('', index=data.index, columns=data.columns)
        for col in data.columns:
            best_idx = data[col].idxmax()
            # for idx in best_idx.values:
            result.loc[best_idx, col] = attr
        return result

    styled_df = m.style.apply(highlight_best, axis=None)
    ltex = styled_df.format(precision=2).to_latex(
        **latex_kwargs
    )
    return ltex

# Results - General retrieval (best contrastive sum checkpoint)

In [21]:
# collect all data
ROOT = "runs"

metrics = collect_all(ROOT, 'best-contrastive-sum')
metrics_baselines = collect_all(ROOT, 'original_checkpoint')    # baseline model
metrics_concat = pd.concat([metrics, metrics_baselines], axis=0, join='outer')

metrics = summarize_metrics(
    metrics_concat,
    training_setup=["with_entities", np.nan],
    finetuning=["disabled", "shallow-vpt-5", np.nan],
    # tok_position=["tok_in_place_multi_prompts", np.nan],
    # tok_position_inference=["in_place", None]
    )

# remove contrastive_sum columns
metrics.drop(columns="contrastive_sum", inplace=True)

# remove data, translator, tok_position, training-setup, loss, lr, tok_position_inference from the multi index
metrics.index = metrics.index.droplevel(['data', 'translator', 'training-setup', 'loss', 'lr'])

# reorder columns to t2i-r@1 t2i-r@5 t2i-r@10 t2i-r@50 contrastive_t2i_sum
metrics = metrics[["t2i-r@1", "t2i-r@5", "t2i-r@10", "t2i-r@50", "contrastive_t2i_sum"]]

# transform in percentage
metrics = metrics * 100

# latex = render_to_latex(
#     metrics, 
#     caption="General Retrieval",
#     clines="skip-last;data",
#     hrules=True,
#     column_format="llllccccc",
#     convert_css=True
# )

# print(latex)

metrics

t2i-r@1  \
model    tok_position                finetuning    inference_config            
idclip   tok_in_place_multi_prompts  disabled      conf5                37.7   
                                                   conf1                37.9   
                                                   conf2                37.7   
                                                   conf4                37.5   
                                                   conf3                37.6   
                                     shallow-vpt-5 conf5                38.8   
                                                   conf1                40.8   
                                                   conf2                38.7   
                                                   conf4                38.7   
                                                   conf3                35.9   
         tok_beginning_multi_prompts disabled      conf5                37.4   
                                                   conf1                37.5   
                                                   conf2                37.3   
                                                   conf4                36.8   
                                                   conf3                38.1   
                                     shallow-vpt-5 conf5                40.3   
                                                   conf1                41.3   
                                                   conf2                40.5   
                                                   conf4                40.5   
                                                   conf3                39.0   
baseline NaN                         NaN           NaN                  23.8   
                                                   NaN                  10.5   

                                                                     t2i-r@5  \
model    tok_position                finetuning    inference_config            
idclip   tok_in_place_multi_prompts  disabled      conf5                66.4   
                                                   conf1                69.1   
                                                   conf2                66.5   
                                                   conf4                66.3   
                                                   conf3                67.0   
                                     shallow-vpt-5 conf5                65.1   
                                                   conf1                69.1   
                                                   conf2                65.4   
                                                   conf4                65.7   
                                                   conf3                63.5   
         tok_beginning_multi_prompts disabled      conf5                65.4   
                                                   conf1                68.0   
                                                   conf2                65.3   
                                                   conf4                64.9   
                                                   conf3                66.4   
                                     shallow-vpt-5 conf5                67.8   
                                                   conf1                70.7   
                                                   conf2                69.1   
                                                   conf4                67.8   
                                                   conf3                67.9   
baseline NaN                         NaN           NaN                  51.5   
                                                   NaN                  48.4   

                                                                     t2i-r@10  \
model    tok_position                finetuning    inference_config             
idclip   tok_in_place_multi_prompts  disabled      conf5         

# Results - Entities retrieval (best contrastive sum checkpoint)

In [22]:
# collect all data
ROOT = "runs"

metrics = collect_all(ROOT, 'best-contrastive-sum', test_config='entities')
metrics_baselines = collect_all(ROOT, 'original_checkpoint', test_config='entities')    # baseline model
metrics_concat = pd.concat([metrics, metrics_baselines], axis=0, join='outer')

metrics = summarize_metrics(
    metrics_concat,
    training_setup=["with_entities", np.nan],
    finetuning=["disabled", "shallow-vpt-5", np.nan],
    # tok_position=["tok_in_place_multi_prompts", np.nan],
    # tok_position_inference=["in_place", None]
    )

# remove all columns containing "entity-r"
metrics = metrics.loc[:, ~metrics.columns.str.contains('entity-r')]
metrics.drop(columns="entities_sum", inplace=True)

# remove data, translator, tok_position, training-setup, loss, lr, tok_position_inference from the multi index
metrics.index = metrics.index.droplevel(['data', 'translator', 'training-setup', 'loss', 'lr'])

# reorder columns to t2i-r@1 t2i-r@5 t2i-r@10 t2i-r@50 contrastive_t2i_sum
metrics = metrics[["entity-kmin-r@1", "entity-kmin-r@5", "entity-kmin-r@10", "entity-kmin-r@50", "entities_kmin_sum", "mAP"]]

# transform in percentage
metrics = metrics * 100

metrics

# latex = render_to_latex(
#     metrics, 
#     caption="Entities Retrieval",
#     clines="skip-last;data",
#     hrules=True,
#     column_format="llllcccccc",
#     convert_css=True
# )

# print(latex)

entity-kmin-r@1  \
model    tok_position                finetuning    inference_config                    
idclip   tok_in_place_multi_prompts  disabled      conf5                        22.6   
                                                   conf2                        20.9   
                                                   conf3                        19.3   
                                                   conf1                        15.9   
                                                   conf4                        21.8   
                                     shallow-vpt-5 conf5                        22.9   
                                                   conf2                        22.1   
                                                   conf3                        21.2   
                                                   conf1                        19.3   
                                                   conf4                        21.5   
         tok_beginning_multi_prompts disabled      conf5                        21.8   
                                                   conf2                        20.7   
                                                   conf3                        20.4   
                                                   conf1                        15.4   
                                                   conf4                        21.8   
                                     shallow-vpt-5 conf5                        22.9   
                                                   conf2                        21.8   
                                                   conf3                        22.6   
                                                   conf1                        21.2   
                                                   conf4                        24.3   
baseline NaN                         NaN           NaN                          12.8   

                                                                     entity-kmin-r@5  \
model    tok_position                finetuning    inference_config                    
idclip   tok_in_place_multi_prompts  disabled      conf5                        11.3   
                                                   conf2                        11.3   
                                                   conf3                        10.4   
                                                   conf1                         9.3   
                                                   conf4                        11.3   
                                     shallow-vpt-5 conf5                        13.6   
                                                   conf2                        12.6   
                                                   conf3                        11.9   
                                                   conf1                        12.3   
                                                   conf4                        13.2   
         tok_beginning_multi_prompts disabled      conf5                        11.8   
                                                   conf2                        12.6   
                                                   conf3                        11.8   
                                                   conf1                        11.1   
                                                   conf4                        11.8   
                                     shallow-vpt-5 conf5                        13.0   
                                                   conf2                        12.4   
                                                   conf3                        11.6   
                                                   conf1                        11.5   
                                                   conf4                        13.7   
baseline NaN                         NaN           NaN                           6.0   

                   

# Results - General retrieval (best entities sum checkpoint)

In [17]:
# collect all data
ROOT = "runs"

metrics = collect_all(ROOT, 'best-entities-sum')
metrics_baselines = collect_all(ROOT, 'original_checkpoint')    # baseline model
metrics_concat = pd.concat([metrics, metrics_baselines], axis=0, join='outer')

metrics = summarize_metrics(
    metrics_concat,
    training_setup=["with_entities", np.nan],
    finetuning=["disabled", "shallow-vpt-5", np.nan],
    # tok_position=["tok_in_place_multi_prompts", np.nan],
    # tok_position_inference=["in_place", None]
    )

# remove contrastive_sum columns
metrics.drop(columns="contrastive_sum", inplace=True)

# remove data, translator, tok_position, training-setup, loss, lr, tok_position_inference from the multi index
metrics.index = metrics.index.droplevel(['data', 'translator', 'training-setup', 'loss', 'lr'])

# reorder columns to t2i-r@1 t2i-r@5 t2i-r@10 t2i-r@50 contrastive_t2i_sum
metrics = metrics[["t2i-r@1", "t2i-r@5", "t2i-r@10", "t2i-r@50", "contrastive_t2i_sum"]]

# transform in percentage
metrics = metrics * 100

# latex = render_to_latex(
#     metrics, 
#     caption="General Retrieval",
#     clines="skip-last;data",
#     hrules=True,
#     column_format="llllccccc",
#     convert_css=True
# )

# print(latex)

metrics

t2i-r@1  \
model    tok_position                finetuning    inference_config            
idclip   tok_in_place_multi_prompts  disabled      conf5                36.1   
                                                   conf1                36.6   
                                                   conf2                36.3   
                                                   conf4                36.2   
                                                   conf3                36.1   
                                     shallow-vpt-5 conf1                40.7   
                                                   conf2                38.3   
                                                   conf3                36.1   
         tok_beginning_multi_prompts disabled      conf5                35.9   
                                                   conf1                36.9   
                                                   conf2                36.4   
                                                   conf4                35.7   
                                                   conf3                37.3   
                                     shallow-vpt-5 conf5                40.2   
                                                   conf1                40.3   
                                                   conf2                40.3   
                                                   conf4                40.4   
                                                   conf3                38.9   
baseline NaN                         NaN           NaN                  23.8   
                                                   NaN                  10.5   

                                                                     t2i-r@5  \
model    tok_position                finetuning    inference_config            
idclip   tok_in_place_multi_prompts  disabled      conf5                63.5   
                                                   conf1                66.9   
                                                   conf2                63.7   
                                                   conf4                63.2   
                                                   conf3                64.0   
                                     shallow-vpt-5 conf1                68.6   
                                                   conf2                65.5   
                                                   conf3                63.5   
         tok_beginning_multi_prompts disabled      conf5                63.3   
                                                   conf1                67.2   
                                                   conf2                64.8   
                                                   conf4                63.6   
                                                   conf3                65.5   
                                     shallow-vpt-5 conf5                67.6   
                                                   conf1                70.1   
                                                   conf2                68.6   
                                                   conf4                67.6   
                                                   conf3                67.6   
baseline NaN                         NaN           NaN                  51.5   
                                                   NaN                  48.4   

                                                                     t2i-r@10  \
model    tok_position                finetuning    inference_config             
idclip   tok_in_place_multi_prompts  disabled      conf5                 74.0   
                                                   conf1                 77.6   
                                                   conf2                 74.4   
                                                   conf4                 73.9   
                                                   conf3     

# Results - Entities retrieval (best entities sum checkpoint)

In [20]:
# collect all data
ROOT = "runs"

metrics = collect_all(ROOT, 'best-entities-sum', test_config='entities')
metrics_baselines = collect_all(ROOT, 'original_checkpoint', test_config='entities')    # baseline model
metrics_concat = pd.concat([metrics, metrics_baselines], axis=0, join='outer')

metrics = summarize_metrics(
    metrics_concat,
    training_setup=["with_entities", np.nan],
    finetuning=["disabled", "shallow-vpt-5", np.nan],
    # tok_position=["tok_in_place_multi_prompts", np.nan],
    # tok_position_inference=["in_place", None]
    )

# remove all columns containing "entity-r"
metrics = metrics.loc[:, ~metrics.columns.str.contains('entity-r')]
metrics.drop(columns="entities_sum", inplace=True)

# remove data, translator, tok_position, training-setup, loss, lr, tok_position_inference from the multi index
metrics.index = metrics.index.droplevel(['data', 'translator', 'training-setup', 'loss', 'lr'])

# reorder columns to t2i-r@1 t2i-r@5 t2i-r@10 t2i-r@50 contrastive_t2i_sum
metrics = metrics[["entity-kmin-r@1", "entity-kmin-r@5", "entity-kmin-r@10", "entity-kmin-r@50", "entities_kmin_sum", "mAP"]]

# transform in percentage
metrics = metrics * 100

metrics

# latex = render_to_latex(
#     metrics, 
#     caption="Entities Retrieval",
#     clines="skip-last;data",
#     hrules=True,
#     column_format="llllcccccc",
#     convert_css=True
# )

# print(latex)

entity-kmin-r@1  \
model    tok_position                finetuning    inference_config                    
idclip   tok_in_place_multi_prompts  disabled      conf5                        19.8   
                                                   conf2                        20.1   
                                                   conf3                        19.8   
                                                   conf1                        16.2   
                                                   conf4                        20.7   
                                     shallow-vpt-5 conf5                        22.9   
                                                   conf2                        22.6   
                                                   conf3                        21.5   
                                                   conf1                        21.5   
                                                   conf4                        24.3   
         tok_beginning_multi_prompts disabled      conf5                        23.5   
                                                   conf2                        23.2   
                                                   conf3                        22.1   
                                                   conf1                        19.6   
                                                   conf4                        23.5   
                                     shallow-vpt-5 conf5                        19.8   
                                                   conf2                        20.7   
                                                   conf3                        21.2   
                                                   conf1                        20.4   
                                                   conf4                        22.6   
baseline NaN                         NaN           NaN                          12.8   

                                                                     entity-kmin-r@5  \
model    tok_position                finetuning    inference_config                    
idclip   tok_in_place_multi_prompts  disabled      conf5                        10.8   
                                                   conf2                        11.1   
                                                   conf3                        10.7   
                                                   conf1                         9.4   
                                                   conf4                        10.7   
                                     shallow-vpt-5 conf5                        13.8   
                                                   conf2                        12.5   
                                                   conf3                        11.4   
                                                   conf1                        12.4   
                                                   conf4                        13.0   
         tok_beginning_multi_prompts disabled      conf5                        11.3   
                                                   conf2                        11.7   
                                                   conf3                        11.4   
                                                   conf1                        10.5   
                                                   conf4                        11.2   
                                     shallow-vpt-5 conf5                        12.4   
                                                   conf2                        11.9   
                                                   conf3                        12.0   
                                                   conf1                        11.7   
                                                   conf4                        12.9   
baseline NaN                         NaN           NaN                           6.0   

                   